In [ ]:
import scipy.linalg as la
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import numpy as np
from helper_functions.grape_functions import *
from main_grape.Grape import Grape
%pylab inline



In [ ]:
def run_GHZ(qubit_num):


    #Defining time scales
    total_time = qubit_num *2.0
    steps = qubit_num * 10



    state_transfer = True
    use_inter_vecs = False
    #Defining H0
    qubit_state_num = 2
    freq_ge = 0#GHz
    g = 2*np.pi*0.1 #GHz

    Q_x   = np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)+np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1)
    Q_y   = (0+1j) *(np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)-np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1))
    sigma_z   = np.diag(np.arange(-1,qubit_state_num,2))
    I_q = np.identity(qubit_state_num)
    
    #print Q_x
    #print Q_y
    #print sigma_z

    g_op = nn_chain_kron(sigma_z, I_q, qubit_num, qubit_state_num)
    
#     g_op_test = np.kron(np.kron(np.kron(Q_z,Q_z),I_q),I_q) + np.kron(np.kron(np.kron(I_q,Q_z),Q_z),I_q) +\
#     np.kron(np.kron(np.kron(I_q,I_q),Q_z),Q_z)
#     print np.allclose(g_op, g_op_test)


#     H0 = kron_all(H_q,qubit_num, I_q) + g * g_op
    H0 = g * g_op
    
    #print H0

    # Defining dressed info
    is_dressed = False
#     w_c, v_c, dressed_id = get_dressed_info(H0)
#     dressed_info = {'dressed_id':dressed_id, 'eigenvectors':v_c, 'eigenvalues':w_c,'is_dressed':is_dressed}
    
    dressed_info = None


    #Defining Concerned states (starting states)

    psi0=[0]

    all_ones_index = 0
    for ii in range (qubit_num):
        all_ones_index = all_ones_index + qubit_state_num**ii
    #Defining states to include in the drawing of occupation
    states_draw_list = [0,all_ones_index]
    states_draw_names = ['00..0','11..1']

    #Defining U (Target)

    if is_dressed:

        gg = get_state_index(0,dressed_id) #[00..0]
        ee = get_state_index(all_ones_index,dressed_id) #[11..1]

        target_vec = np.zeros(qubit_state_num**qubit_num,dtype=complex)
        target_vec = target_vec +  (1)/np.sqrt(2)*v_c[:,gg] + (1)/np.sqrt(2)*v_c[:,ee]

        U = [target_vec]

    else:   
        target_vec = np.zeros(qubit_state_num**qubit_num,dtype=complex)
        target_vec[0]=(1)/np.sqrt(2)
        target_vec[all_ones_index]=1/np.sqrt(2)

        U = [target_vec]
    #Defining control Hs

    Hops = []
    Hnames = []
    ops_max_amp = []
    max_amp=2*np.pi*0.5
    Hops,Hnames,ops_max_amp = append_separate_krons(Q_x,'x',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=max_amp)
    Hops,Hnames,ops_max_amp = append_separate_krons(Q_y,'y',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=max_amp)
    #Hops,Hnames,ops_max_amp = append_separate_krons(Q_y,'z',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=max_amp)
    
    #print Hops[0]
    print Hnames

    #Defining convergence parameters
    max_iterations = 100
    decay = 1000 #max_iterations/2
    convergence = {'rate':0.01, 'update_step':10 ,'max_iterations':max_iterations,\
                   'conv_target':1e-3,'learning_rate_decay':decay,'min_grad':1e-60}

    reg_coeffs = {}
    
    #import h5py
    #with h5py.File('/home/nelson/Simulations/GRAPE-Data/spin_chain_hadamard/00002_gpu_spin_chain_hadamard_N9.h5','r') as hf:
    #    u0 = np.array(hf.get('uks'))[-1]
    
    u0=None

    uks,U_final = Grape(H0,Hops,Hnames,U,total_time,steps,psi0,convergence=convergence, draw = [states_draw_list,states_draw_names],  

                        show_plots = False, use_gpu = False,sparse_H=False,sparse_U=False,state_transfer = state_transfer, use_inter_vecs = use_inter_vecs,
           unitary_error = 1e-8,  maxA=ops_max_amp,Taylor_terms =[20,0],initial_guess=u0,
                       dressed_info = dressed_info, method = 'ADAM', reg_coeffs=reg_coeffs, file_name='dense_cpu_spin_chain_ghz_N%d' %(qubit_num), 
                        data_path = '/home/nelson/Simulations/GRAPE-Data/spin_chain_ghz')

In [ ]:
run_GHZ(qubit_num=11)